### © SATYAJIT GHOSH (https://github.com/SATYAJIT1910)

In [2]:
'''
Requirements

sudo apt install tesseract-ocr-ben
sudo apt-get install sqlite3 libsqlite3-dev  
sudo apt-get install nltk

pip install pdf2image
pip install openpyxl
pip install pandas
pip install pytesseract
pip install opencv-python

'''


#importing libraries
from pdf2image import convert_from_path
import nltk as nl
import os
import sqlite3
import cv2 
import pytesseract
import pandas as pd
import re

mydb = sqlite3.connect('corpus.db') #Connects to database

# CORPUS CREATION

THIS CODE WILL MAKE EVERY PAGE OF THE DOWNLOADED PDF FILES A IMAGE AND STORES IT TO THE RESPECTIVE FOLDER FOR OCR OPERATION

In [ ]:
# This method saves every page of PDF document as Image
def save_image(k):
    images = convert_from_path('BOOKS/'+str(k)+'.pdf')
    path='IMAGE/'+str(k) # Path to store the Image files
    for i in range(len(images)):
        if not(os.path.isdir(path)):
            os.mkdir(path) # Creates the directory
        # Save Images
        images[i].save(path+'/'+str(k)+'_'+ str(i+1) +'.jpg', 'JPEG')

for k in range(1,201):
    save_image(k) #Function Call

In [ ]:
def nomalization(text):
    text=only_bengali_characters(text)
    return text

def only_bengali_characters(text):
    '''
    All the bengali unicode characters are present from u980 to u09FF .
    Here we are using regex to remove all the invaild unicode characters from our output .
    We are also removing whitespaces .
    '''
    return re.sub('[^\u0980-\u09FF ]',r'',text)

THIS CODE WILL PERFORM OCR OPERATION AND STORE THE EXTRACTED TEXTS IN THE DATABASE

In [ ]:
mydb.execute("CREATE TABLE BOOK(ID INTEGER AUTO_INCREMENT PRIMARY KEY,AUTHOR TEXT,TITLE TEXT);")
mydb.execute("CREATE TABLE PAGE(ID INTEGER ,PAGENO INTEGER,CONTENT TEXT,FOREIGN KEY (ID) REFERENCES BOOK(ID));")


# This method performs OCR operation and saves the result on the database
def perform_OCR_and_save(path,ID,pageno):
	# Reading the Image
	img = cv2.imread(path)
	# Adding custom options
	custom_config = r'--oem 3 --psm 6'
	text=pytesseract.image_to_string(img, config=custom_config,lang='ben') #OCR operation

	# normalizing the texts
	text=nomalization(text)

	QUERY="INSERT INTO PAGE VALUES("+ID+","+pageno+",'"+text+"');"
	#print(QUERY)
	mydb.execute(QUERY) # Stores the extracted text to the database



# assign directory
directory = 'IMAGE'

# iterate over files in
# that directory

for filename in os.listdir(directory):
	f = os.path.join(directory, filename)
	print("LOG: OCR WORKING ON DIR : "+f)
	ID=f.split("/")[1] # Removing the Book ID from the directory itself
	QUERY='INSERT INTO BOOK(ID) VALUES('+ID+');' # Inserting the Book ID only to Database
	mydb.execute(QUERY)

	log=0 # for logging the progress
	for images in os.listdir(f):
		i=os.path.join(f,images)
		PAGENO=i.split("_")[-1]
		PAGENO=PAGENO[:-4] # Removing the Page No from the directory itself
		perform_OCR_and_save(i,str(ID),str(PAGENO))
		print("LOG : "+str(log)+" "+str(ID)+" "+str(PAGENO)+" DONE")
		log=log+1
mydb.commit() # Commits to database

THIS CODE PERFORM ENTRIES OF AUTHOR AND TITLE OF THE BOOK FROM THE EXCEL FILE DATA TO DATABASE

In [ ]:
df = pd.read_excel (r'RECORDS_200.xlsx')


for i in range(len(df)):
    author=df["AUTHOR"][i]
    title=df["TITLE"][i]


    QUERY="UPDATE BOOK SET AUTHOR = '" + str(author) +"'"+" WHERE ID =" + str(i+1)+";"
    mydb.execute(QUERY)

    QUERY="UPDATE BOOK SET TITLE = '" + str(title) +"'"+" WHERE ID =" + str(i+1)+";"
    mydb.execute(QUERY)
mydb.commit()

# PLAGIARISM CHECKING

In [3]:
def levenshtein(text1,text2):
  diff=nl.edit_distance(text1,text2)
  # FORMULA
  result=(1-(diff/max(len(text1),len(text2))))*100
  result="{:.2f}".format(result) #taking two decimal place value only

  return float(result)

In [4]:
# source : https://github.com/stopwords-iso/stopwords-bn

In [5]:
bangla_stopwords=["অতএব",
"অথচ",
"অথবা",
"অনুযায়ী",
"অনেক",
"অনেকে",
"অনেকেই",
"অন্তত",
"অন্য",
"অবধি",
"অবশ্য",
"অর্থাত",
"আই",
"আগামী",
"আগে",
"আগেই",
"আছে",
"আজ",
"আপনার",
"আপনি",
"আবার",
"আমরা",
"আমাকে",
"আমাদের",
"আমার",
"আমি",
"আর",
"আরও",
"ই",
"ইত্যাদি",
"ইহা",
"উচিত",
"উত্তর",
"উনি",
"উপর",
"উপরে",
"এ",
"এঁদের",
"এঁরা",
"এই",
"একই",
"একটি",
"একবার",
"একে",
"এক্",
"এখন",
"এখনও",
"এখানে",
"এখানেই",
"এটা",
"এটাই",
"এটি",
"এত",
"এতটাই",
"এতে",
"এদের",
"এব",
"এবং",
"এবার",
"এমন",
"এমনকী",
"এমনি",
"এর",
"এরা",
"এল",
"এস",
"এসে",
"ঐ",
"ও",
"ওঁদের",
"ওঁর",
"ওঁরা",
"ওই",
"ওকে",
"ওখানে",
"ওদের",
"ওর",
"ওরা",
"কখনও",
"কত",
"কবে",
"কমনে",
"কয়েক",
"কয়েকটি",
"করছে",
"করছেন",
"করতে",
"করবে",
"করবেন",
"করলে",
"করলেন",
"করা",
"করাই",
"করায়",
"করার",
"করি",
"করিতে",
"করিয়া",
"করিয়ে",
"করে",
"করেই",
"করেছিলে",
"করেছে",
"করেছেন",
"করেন",
"কাউকে",
"কাছ",
"কাছে",
"কাজ",
"কাজে",
"কারও",
"কারণ",
"কি",
"কিংবা",
"কিছু",
"কিছুই",
"কিন্তু",
"কী",
"কে",
"কেউ",
"কেউই",
"কেখা",
"কেন",
"কোটি",
"কোন",
"কোনও",
"কোনো",
"ক্ষেত্রে",
"কয়েক",
"খুব",
"গিয়ে",
"গিয়েছে",
"গিয়ে",
"গুলি",
"গেছে",
"গেল",
"গেলে",
"গোটা",
"চলে",
"চান",
"চায়",
"চার",
"চালু",
"চেয়ে",
"চেষ্টা",
"ছাড়া",
"ছাড়াও",
"ছিল",
"ছিলেন",
"জন",
"জনকে",
"জনের",
"জন্য",
"জানতে",
"জানা",
"জানানো",
"জানায়",
"জানিয়ে",
"জানিয়েছে",
"জে",
"জ্নজন",
"টি",
"ঠিক",
"তখন",
"তত",
"তথা",
"তবু",
"তবে",
"তা",
"তাঁকে",
"তাঁদের",
"তাঁর",
"তাঁরা",
"তাঁাহারা",
"তাই",
"তাও",
"তাকে",
"তাতে",
"তাদের",
"তার",
"তারপর",
"তারা",
"তারৈ",
"তাহলে",
"তাহা",
"তাহাতে",
"তাহার",
"তিনঐ",
"তিনি",
"তিনিও",
"তুমি",
"তুলে",
"তেমন",
"তো",
"তোমার",
"থাকবে",
"থাকবেন",
"থাকা",
"থাকায়",
"থাকে",
"থাকেন",
"থেকে",
"থেকেই",
"থেকেও",
"দিকে",
"দিতে",
"দিন",
"দিয়ে",
"দিয়েছে",
"দিয়েছেন",
"দিলেন",
"দু",
"দুই",
"দুটি",
"দুটো",
"দেওয়া",
"দেওয়ার",
"দেওয়া",
"দেখতে",
"দেখা",
"দেখে",
"দেন",
"দেয়",
"দ্বারা",
"ধরা",
"ধরে",
"ধামার",
"নতুন",
"নয়",
"না",
"নাই",
"নাকি",
"নাগাদ",
"নানা",
"নিজে",
"নিজেই",
"নিজেদের",
"নিজের",
"নিতে",
"নিয়ে",
"নিয়ে",
"নেই",
"নেওয়া",
"নেওয়ার",
"নেওয়া",
"নয়",
"পক্ষে",
"পর",
"পরে",
"পরেই",
"পরেও",
"পর্যন্",
"পাওয়া",
"পাচ",
"পারি",
"পারে",
"পারেন",
"পি",
"পেয়ে",
"পেয়্র্",
"প্রতি",
"প্রথম",
"প্রভৃতি",
"প্রাথমিক",
"প্রায়",
"প্রায়",
"ফলে",
"ফিরে",
"ফের",
"বদলে",
"বন",
"বরং",
"বলতে",
"বলল",
"বললেন",
"বলা",
"বলে",
"বলেছেন",
"বলেন",
"বসে",
"বহু",
"বা",
"বাদে",
"বার",
"বি",
"বিনা",
"বিভিন্ন",
"বিশেষ",
"বিষয়টি",
"বেশ",
"বেশি",
"ব্যবহার"
"ব্যাপারে",
"ভাবে",
"ভাবেই",
"মতো",
"মতোই",
"মধ্যভাগে"
"মধ্যে",
"মধ্যেই",
"মধ্যেও",
"মনে",
"মাত্র",
"মাধ্যমে",
"মোট",
"মোটেই",
"যখন",
"যত",
"যতটা",
"যথেষ্ট",
"যদি",
"যদিও",
"যা",
"যাঁর",
"যাঁরা",
"যাওয়া",
"যাওয়ার",
"যাওয়া",
"যাকে",
"যাচ্ছে",
"যাতে",
"যাদের",
"যান",
"যাবে",
"যায়",
"যার",
"যারা",
"যিনি",
"যে",
"যেখানে",
"যেতে",
"যেন",
"যেমন",
"র",
"রকম",
"রয়েছে",
"রাখা",
"রেখে",
"লক্ষ",
"শুধু",
"শুরু",
"সঙ্গে",
"সঙ্গেও",
"সব",
"সবার",
"সমস্ত",
"সম্প্রতি"
"সহ",
"সহিত",
"সাধারণ",
"সামনে",
"সি",
"সুতরাং",
"সে",
"সেই",
"সেখান",
"সেখানে",
"সেটা",
"সেটাই",
"সেটাও",
"সেটি",
"স্পষ্ট",
"স্বয়ং",
"হইতে",
"হইবে",
"হইয়া",
"হওয়া",
"হওয়ায়",
"হওয়ার",
"হচ্ছে",
"হত",
"হতে",
"হতেই",
"হন",
"হবে",
"হবেন",
"হয়",
"হয়তো",
"হয়নি",
"হয়ে",
"হয়েই",
"হয়েছিল",
"হয়েছে",
"হয়েছেন",
"হল",
"হলে",
"হলেই",
"হলেও",
"হলো",
"হাজার",
"হিসাবে",
"হৈলে",
"হোক",
"হয়"]

In [6]:
# This method traverse through the entire database and searches for similier texts and gives the output 
def check(inputText):
    scorelist=[]
    inputTextToken=inputText.split(" ")
    inputTextToken = [i for i in inputTextToken if i not in bangla_stopwords]
    
    for bookID in range(1,201):
        #print("Starting work on ",bookID)
        mycursor=mydb.execute("SELECT MAX(PAGENO) FROM PAGE WHERE ID="+str(bookID)+";")
        totalpages=int(mycursor.fetchone()[0])
        for page in range(1,totalpages+1):
            mycursor=mydb.execute("SELECT CONTENT FROM PAGE WHERE ID="+str(bookID)+" AND PAGENO="+str(page)+";")
            dbtext=mycursor.fetchone()[0]
            
            dbtokens=dbtext.split(" ")
            dbtokens = [i for i in dbtokens if i not in bangla_stopwords] #Removing stopwords

            #gives the similarity score
            score=levenshtein(dbtokens,inputTextToken)
            if(score>=20):
                scorelist.append([bookID,page,score])
    scorelist.sort(key = lambda y: y[2])
    return scorelist

In [7]:
# This method accepts the list from the check() method and finds out the title and author information from the database
def findTitle(result):
    final=[]
    for i in range(0,len(result)):
        bookID=result[i][0]
        pageno=result[i][1]
        score=result[i][2]
        
        mycursor=mydb.execute("SELECT * FROM BOOK WHERE ID="+str(bookID))
        dboutput=mycursor.fetchone()
        
        author=dboutput[1]
        title=dboutput[2]
        
        final.append([bookID,author,title,pageno,score])
    return final
        
    

## Driver Codes

In [21]:
inputText='''

সোনা বললে, “কতদিন পালিয়ে বেড়াবে? বাড়ি যাবে না? তোমার মা নেই? ঘড়িওলা হাউ হাউ করে কীদতে লাগল। আছে গো, আছে, সব আছে; বড়ো ভালো সরুচাকলি বানায় আমার মা, একবার খেলে
আর ভোলা যায় না। কবে যে আবার তাকে দেখতে পাবো! টিয়া বললে,দুষ্টু মাকু থাকগে পড়ে, তুমি মার কাছে ফিরে যাও ।” এই বলে পুটলির কোণা দিয়ে টিয়া চোখ মুছল। ঘড়িওলাও চোখ মুছল। “তাই কি হয়, দিদি, মাকু যে আমার প্রাণ, ওকে নাগালের বাইরে যেতে দিই কী করে? ওর চাবি ফুরিয়ে গেলেই যে নেতিয়ে পড়বে, তখন চোর ডাকাতে ওর কলকজা খুলে নিলেই মাকুর আমার হয়ে গেল।” “কবে চাবি ফুরুবে?” এক বছরের চাবি দেওয়া, তার সাড়ে এগারো মাস কেটে গেছে, আর পনেরো দিন। বলো,
ওকে বের করে চোখে চোখে রাখবে? সোনা বললে, “সেলাই কল চালায় আর নিজের পেটের চাবিটা ঘুরিয়ে নিতে পারবে না? ঘড়িওলা ব্যস্ত হয়ে উঠল। পেটে নয়, দিদি, পেটে নয়, পিটের মধ্যিখানে, গায়ে-বসা এতটুকু চাবি, কানখুসকি দিয়ে ঘুরুতে হয়। নইলে মাকু যা দস্যি, কৰে টেনে খুলে ফেলে দিত। ওখানে সে হাত পায় না, হাত দুটো ইচ্ছা করে একটু বেঁটে করে দিয়েছি। কথা বলতে বলতে কখন তারা শুনশুনির মাঠ পেরিয়ে এসেছে, সামনে দেখে ঘন বন। বনের মধ্যে খানিক রোদ, খানিক ছায়া, পাখির ডাক, পাতার খসখস, বুনো ফুলের আর ধূপ কাঠের গন্ধ। ঘড়িওলা বললে,আমি আর যাব না, মাকু আমাকে দেখলে ছেঁকে ধরবে, আমার ভয় করে। তোমরা স্কুলে ভর্তি হয়েছ, ভয় পাও না, তোমরা যাও! আমি এখানে গাছের মাথায় পাতার ঘর বেঁধে অপেক্ষা করি।' এই বলে ঘড়িওলা সোনা-টিয়ার ঘাড় ধরে একটু ঠেলে দিল।
দুই ঠেলা খেয়ে প্রায় একরকম ঢুকেই গেছিল বনের মধ্যে সোনা আর টিয়া, এমন সময় ঘড়িওলা পেছন
থেকে ডেকে বলল, চললে কোথায়? হ্যান্ডবিল নিতে হবে না? তা নইলে মাকুর বিষয়ে বিশেষ
বিজ্ঞপ্তি জানবে কী করে? বলি, তাকে চিনতে হবে তো? এই বলে ঝোলা থেকে একটা বড়ো মতো গোলাপি কাগজ বের করে, পাশের মাটির টিপির
ওপরে চড়ে গলা খাঁকরে পড়তে লাগল  মাত্র পচিশ পয়সায়! অদ্ভুত! অত্যান্চর্য  মাকু দি গ্রেট! অভাবনীয় দৃশ্য দেখে যান!


'''

In [22]:
if(len(inputText)<=300):
    print("Minimum 300 Characters required for accurate outputs")
elif(len(inputText)>5000):
    print("To large to handle,should be within 5000 characters")
else:
    finaloutput=findTitle(check(inputText))
    

In [23]:
finaloutput

[[150, 'মজুমদার, লীলা', 'মাকু', 12, 49.79]]

In [24]:
mydb.close() #closes the database

Thank You